Resultados de la primera vuelta

In [1]:
import pandas as pd
import numpy as np
import pyreadstat as pyd

In [2]:
# datos de la primera vuelta electoral
primeravuelta, meta1 = pyd.read_sav('Data/Primera-Vuelta.sav', apply_value_formats=True)

In [3]:
primeravuelta.columns

Index(['DIGNIDAD_CODIGO', 'DIGNIDAD_NOMBRE', 'PROVINCIA_CODIGO',
       'PROVINCIA_NOMBRE', 'CIRCUNSCRIPCION_CODIGO', 'CIRCUNSCRIPCION_NOMBRE',
       'CANTON_CODIGO', 'CANTON_NOMBRE', 'PARROQUIA_CODIGO',
       'PARROQUIA_NOMBRE', 'JUNTA_SEXO', 'OP_CODIGO', 'CANDIDATO_NOMBRE',
       'SUFRAGANTES', 'BLANCOS', 'NULOS', 'VOTOS'],
      dtype='object')

In [4]:
# Ajustar tipo de datos - convertir float primero, luego int
primeravuelta = primeravuelta.astype({
    'DIGNIDAD_NOMBRE': str,
    'PROVINCIA_CODIGO': float,  # Primero a float
    'PROVINCIA_NOMBRE': str,
    'CIRCUNSCRIPCION_CODIGO': str,
    'CIRCUNSCRIPCION_NOMBRE': str,
    'CANTON_CODIGO': float,  # Primero a float
    'CANTON_NOMBRE': str, 
    'PARROQUIA_CODIGO': float,  # Primero a float
    'PARROQUIA_NOMBRE': str,
    'JUNTA_SEXO': str,
    'OP_CODIGO': float,  # Primero a float
    'SUFRAGANTES': float, 
    'BLANCOS': float,
    'NULOS': float, 
    'VOTOS': float,
    'CANDIDATO_NOMBRE': str
})

# Luego convertir a int (sin decimales)
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'OP_CODIGO']] = \
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'OP_CODIGO']].astype(int)

In [5]:
primeravuelta.head(5)

,DIGNIDAD_CODIGO,DIGNIDAD_NOMBRE,PROVINCIA_CODIGO,PROVINCIA_NOMBRE,CIRCUNSCRIPCION_CODIGO,CIRCUNSCRIPCION_NOMBRE,CANTON_CODIGO,CANTON_NOMBRE,PARROQUIA_CODIGO,PARROQUIA_NOMBRE,JUNTA_SEXO,OP_CODIGO,CANDIDATO_NOMBRE,SUFRAGANTES,BLANCOS,NULOS,VOTOS
0,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,7,LEONIDAS IZA,9755.0,188.0,804.0,574.0
1,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,18,HENRY CUCALON,9755.0,188.0,804.0,39.0
2,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,32,HENRY KRONFLE KOZHAYA,9755.0,188.0,804.0,34.0
3,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,34,ANDREA GONZALEZ,9755.0,188.0,804.0,433.0
4,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,39,PEDRO GRANJA,9755.0,188.0,804.0,56.0


In [6]:
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'PARROQUIA_NOMBRE']].head(5)

,PROVINCIA_CODIGO,CANTON_CODIGO,PARROQUIA_CODIGO,PARROQUIA_NOMBRE
0,1,260,285,BAÑOS
1,1,260,285,BAÑOS
2,1,260,285,BAÑOS
3,1,260,285,BAÑOS
4,1,260,285,BAÑOS


In [7]:
dic = pd.read_excel('data/diccionario_parroquias.xlsx')

In [8]:
# Renombrar columnas para que coincidan con primeravuelta
dic = dic.rename(columns={'CODPRO': 'PROVINCIA_CODIGO',
                          'CODCAN': 'CANTON_CODIGO',
                          'CODPAR': 'PARROQUIA_CODIGO'}
                ).astype({'PROVINCIA_CODIGO': int, 
                          'CANTON_CODIGO': int,
                          'PARROQUIA_CODIGO': int
                          })

In [9]:
primeravuelta = primeravuelta.merge(
    dic,
    on = ['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO'], how='left'
    )

De momento, exploraremos solo los votos presidenciales

In [10]:
# Filtrar datos para PRESIDENTE Y VICEPRESIDENTE
presidente1 = primeravuelta[primeravuelta['DIGNIDAD_NOMBRE'] == 'PRESIDENTE Y VICEPRESIDENTE']

Los resultados de sufragantes, nulos y blancos solo pueden obtenerse a nivel de parroquia y junta de sexo. Además, hay que recordar que como el mapa del CNE es más viejo, hay que tratar la base para luego homologarla con el censo del 2022.

Observa el ejemplo siguiente.

In [11]:
# Así deben desagregarse los datos
presidente1[presidente1['ADM3_PCODE'] == 'EC010150']\
    .groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO'], as_index=False)\
    .agg(Sufragantes=('SUFRAGANTES', 'mean'),
         Votos_válidos=('VOTOS', 'sum'),
         Nulos=('NULOS', 'mean'),
         Blancos=('BLANCOS', 'mean')
        )

,ADM3_PCODE,PARROQUIA_NOMBRE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos
0,EC010150,MACHANGARA,FEMENINO,4831.0,4492.0,288.0,48.0
1,EC010150,MACHANGARA,MASCULINO,4184.0,3877.0,241.0,63.0


In [12]:
# exportar para verificar sumas:
# presidente1[presidente1['PARROQUIA_NOMBRE'] == 'CAÑARIBAMBA'].to_excel("data/comprobaciones/canaribamba.xlsx", index=False)

Aun así hay leves descuadres en las sumas de válidos, blancos y nulos. Pero eso ya es cuestión de la fuente. De momento, vamos a tratar los datos como sigue:

In [ ]:
resumen1 = resumen1.groupby(['ADM3_PCODE', 'JUNTA_SEXO'], as_index=False)\
    .agg(Sufragantes=('Sufragantes', 'mean'),
         Votos_válidos=('Votos_válidos', 'sum'),
         Nulos=('Nulos', 'mean'),
         Blancos=('Blancos', 'mean')
         )

# Obtener resultados por parroquia
resumen1 = resumen1.groupby(['ADM3_PCODE'], as_index=False)\
    .agg(Sufragantes=('Sufragantes', 'sum'),
         Votos_válidos=('Votos_válidos', 'sum'),
         Nulos=('Nulos', 'sum'),
         Blancos=('Blancos', 'sum')
         )

resumen1.head(10)

,ADM3_PCODE,Sufragantes,Votos_válidos,Nulos,Blancos
0,EC010150,9015.0,8369.0,529.0,111.0
1,EC010151,18049.0,16323.0,1404.0,312.0
2,EC010152,4073.0,3640.0,321.0,112.0
3,EC010153,1006.0,860.0,83.0,63.0
4,EC010154,1659.0,1447.0,162.0,48.0
5,EC010155,2947.0,2591.0,270.0,84.0
6,EC010156,3166.0,2813.0,284.0,68.0
7,EC010157,4490.0,3898.0,432.0,158.0
8,EC010158,4302.0,3899.0,315.0,87.0
9,EC010159,1276.0,1072.0,126.0,78.0


In [14]:
# Calcular porcentajes de nulos y blancos
resumen1['Nulos_pct'] = resumen1['Nulos'] / resumen1['Sufragantes']
resumen1['Blancos_pct'] = resumen1['Blancos'] / resumen1['Sufragantes']
resumen1.head(10)

,ADM3_PCODE,Sufragantes,Votos_válidos,Nulos,Blancos,Nulos_pct,Blancos_pct
0,EC010150,9015.0,8369.0,529.0,111.0,0.058680,0.012313
1,EC010151,18049.0,16323.0,1404.0,312.0,0.077788,0.017286
2,EC010152,4073.0,3640.0,321.0,112.0,0.078812,0.027498
3,EC010153,1006.0,860.0,83.0,63.0,0.082505,0.062624
4,EC010154,1659.0,1447.0,162.0,48.0,0.097649,0.028933
5,EC010155,2947.0,2591.0,270.0,84.0,0.091619,0.028504
6,EC010156,3166.0,2813.0,284.0,68.0,0.089703,0.021478
7,EC010157,4490.0,3898.0,432.0,158.0,0.096214,0.035189
8,EC010158,4302.0,3899.0,315.0,87.0,0.073222,0.020223
9,EC010159,1276.0,1072.0,126.0,78.0,0.098746,0.061129


# Distribución de votos a candidatos por sufragante

In [15]:
# Agregamos clasificaciones ideológicas de candidatos
Political_Compass = pd.read_excel('data/PiliticalCompass.xlsx')
Political_Compass.head(3)

,Candidato,CANDIDATO_NOMBRE,Eje Económico (Izq–Der),Eje Social (Lib–Aut),Clasificación,Justificación breve,Separación
0,Luisa González (RC),LUISA GONZALEZ,–2.0,+1.5,Izquierda autoritaria,"Estado fuerte, redistribución, liderazgo centr...",LUISA GONZALEZ\n(Izquierda autoritaria)
1,Jorge Escala (Unidad Popular),JORGE ESCALA,–2.5,+2.0,Izquierda autoritaria,"Marxismo sindical, estatismo, orden desde movi...",Izquierda autoritaria
2,Pedro Granja (PSE),PEDRO GRANJA,–2.0,+1.0,Izquierda autoritaria,"Socialismo democrático radical, impuestos a gr...",Izquierda autoritaria


In [16]:
presidente1clas = presidente1.merge(Political_Compass[['CANDIDATO_NOMBRE', 'Separación']], on='CANDIDATO_NOMBRE', how='left')

In [17]:
# Hay que hacer un doble tratamiento igual que antes (por si hubiera más de una parroquia por ADM3_PCODE)
distribucioncandidatos  = \
    presidente1clas.groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'Separación'], 
                            as_index=False)\
                                .agg(Sufragantes=('SUFRAGANTES', 'sum'), 
                                     Votos_candidato=('VOTOS', 'sum'))

In [18]:
distribucioncandidatos[distribucioncandidatos['Separación'] == 'Centro']

,ADM3_PCODE,PARROQUIA_NOMBRE,Separación,Sufragantes,Votos_candidato
0,EC010150,MACHANGARA,Centro,9015.0,436.0
14,EC010151,BAÑOS,Centro,18049.0,805.0
28,EC010152,CUMBE,Centro,4073.0,122.0
42,EC010153,CHAUCHA / ANGAS,Centro,1006.0,26.0
56,EC010154,CHECA JIDCAY,Centro,1659.0,63.0
...,...,...,...,...,...
14392,EC240156,SAN JOSE DE ANCON,Centro,4556.0,124.0
14406,EC240250,LA LIBERTAD,Centro,76511.0,2294.0
14420,EC240350,GRAL. A. E. GALLO,Centro,2144.0,56.0
14434,EC240351,ANCONCITO,Centro,9073.0,140.0


Existen descuadres en los resultados. Habría que considerarlo.

In [19]:
distribucioncandidatos2  = \
    distribucioncandidatos.groupby(['ADM3_PCODE', 'Separación', 'JUNTA_SEXO'], 
                            as_index=False)\
                                .agg(Sufragantes=('Sufragantes', 'mean'), 
                                     Votos_candidato=('Votos_candidato', 'sum'))

KeyError: 'JUNTA_SEXO'

In [ ]:
distribucioncandidatos2.head()
# Comprobar con distribucioncandidatos2.to_excel("data/comprobaciones/distribucioncandidatos_ec010150_sexo.xlsx", index=False)
# Recuerda filtrar un ADM3_PCODE

,ADM3_PCODE,Separación,JUNTA_SEXO,Sufragantes,Votos_candidato
0,EC010150,Centro,FEMENINO,4831.0,242.0
1,EC010150,Centro,MASCULINO,4184.0,194.0
2,EC010150,Centro con tinte\ninstitucionalista,FEMENINO,4831.0,7.0
3,EC010150,Centro con tinte\ninstitucionalista,MASCULINO,4184.0,9.0
4,EC010150,Centro pragmático,FEMENINO,4831.0,16.0


In [ ]:
distribucioncandidatos2['Votos_candidato_pct'] = distribucioncandidatos2['Votos_candidato'] / distribucioncandidatos2['Sufragantes']

In [ ]:
distribucioncandidatos2.head(4)

,ADM3_PCODE,Separación,JUNTA_SEXO,Sufragantes,Votos_candidato,Votos_candidato_pct
0,EC010150,Centro,FEMENINO,4831.0,242.0,0.050093
1,EC010150,Centro,MASCULINO,4184.0,194.0,0.046367
2,EC010150,Centro con tinte\ninstitucionalista,FEMENINO,4831.0,7.0,0.001449
3,EC010150,Centro con tinte\ninstitucionalista,MASCULINO,4184.0,9.0,0.002151


In [ ]:
reestructurado = distribucioncandidatos2.\
    pivot(index=['ADM3_PCODE', 'JUNTA_SEXO'], columns=['Separación'], 
          values='Votos_candidato_pct').fillna(0).reset_index()

In [ ]:
# Unir tablas
table_resumen1 = resumen1.merge(reestructurado, on=['ADM3_PCODE', 'JUNTA_SEXO'], how='left').fillna(0)

In [ ]:
# Separamos por sexo y exportamos (porque será una estructura más manejable en Tableau)